# Vectorization

In [ ]:
!pip install transformers

In [2]:
import pandas as pd
from transformers import BertTokenizer, BertModel
import torch

In [3]:
df = pd.read_csv('preprocessed_data/Combined_Data.csv')
df.head()

,ASIN,MARKETPLACE,REGION,BRAND_NAME,combined_string_full,combined_string_categories,ORGANIC_UNITS_SOLD,SALE_PRICE_EUR
0,B06X6G3FP1,ES,EU,Espring,"Arts, Crafts & Sewing. Painting, Drawing & Art...","Arts, Crafts & Sewing. Painting, Drawing & Art...",1.000000,22.790000
1,B017GLNHEG,IT,EU,H&S,Tools & Home Improvement. Hardware. Wheels. P...,Tools & Home Improvement. Hardware. Wheels,1.446809,16.744681
2,B098XCZZ3Q,ES,EU,Espring,Home & Kitchen. Kitchen & Dining. Pastry utens...,Home & Kitchen. Kitchen & Dining. Pastry utensils,0.000000,16.990000
3,B07GTF236R,US,US,H&S,Sports & Outdoors. Exercise & Fitness. Straps....,Sports & Outdoors. Exercise & Fitness. Straps,1.737500,5.441480
4,B0857JT2ZP,DE,EU,Espring,"Arts, Crafts & Sewing. Painting, Drawing & Art...","Arts, Crafts & Sewing. Painting, Drawing & Art...",0.333333,10.990000


### Full


In [4]:
#takes ~20min to execute.

from transformers import BertTokenizer, BertModel
import torch

#load pre-trained model tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

#load pre-trained model
model = BertModel.from_pretrained('bert-base-uncased')

#make sure model is in evaluation mode (important for models with dropout layers)
model.eval()

#list of sentences
sentences = list(df['combined_string_full'])

#initialize a list that will hold the sentence embeddings
sentence_embeddings = []

#iterate over sentences and compute embeddings
for sentence in sentences:
    #encode text input
    encoded_input = tokenizer(sentence, return_tensors='pt')

    #forward pass, get hidden states
    with torch.no_grad():
        outputs = model(**encoded_input)

    #only take the output embeddings from the last layer
    last_hidden_states = outputs.last_hidden_state

    #use the mean of the last layer hidden states as the sentence embedding
    sentence_embedding = torch.mean(last_hidden_states, dim=1).squeeze().numpy()

    #append the sentence embedding to the list
    sentence_embeddings.append(sentence_embedding)


print(len(sentence_embeddings))
df['vectorized_output_full'] = sentence_embeddings
df.to_parquet('preprocessed_data/Vectorized_Data_Full.parquet')
df.to_csv('preprocessed_data/Vectorized_Data_Full.csv',index=False) #save in csv for easier visualization of the file locally

(…)cased/resolve/main/tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

(…)bert-base-uncased/resolve/main/vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

(…)base-uncased/resolve/main/tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

(…)rt-base-uncased/resolve/main/config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

1165


### Categories

In [5]:
from transformers import BertTokenizer, BertModel
import torch

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

model = BertModel.from_pretrained('bert-base-uncased')
model.eval()

sentences = list(df['combined_string_categories'])
sentence_embeddings = []

for sentence in sentences:

    encoded_input = tokenizer(sentence, return_tensors='pt')

    with torch.no_grad():
        outputs = model(**encoded_input)

    last_hidden_states = outputs.last_hidden_state
    sentence_embedding = torch.mean(last_hidden_states, dim=1).squeeze().numpy()
    sentence_embeddings.append(sentence_embedding)

print(len(sentence_embeddings))
df['vectorized_output_categories'] = sentence_embeddings
df.to_parquet('preprocessed_data/Vectorized_Data_Categories.parquet')
df.to_csv('preprocessed_data/Vectorized_Data_Categories.csv',index=False)

1165


In [6]:
#for easy download
from google.colab import files
files.download('/content/preprocessed_data/Vectorized_Data_Categories.csv')
files.download('/content/preprocessed_data/Vectorized_Data_Full.csv')
files.download('/content/preprocessed_data/Vectorized_Data_Categories.parquet')
files.download('/content/preprocessed_data/Vectorized_Data_Full.parquet')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>